In [ ]:
import hdf5storage
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from analysis_util import *
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib.ticker import MaxNLocator
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import pickle

import seaborn as sns
'''this script test the hypothesis whether people respond to a fixed bound
pepole  '''
# global setting f
plt.rcParams.update({"font.size": 18})
colors = sns.color_palette("deep")
sns.set(style="whitegrid")
# top level dir 
# 





In [ ]:

PROJECT_DIR = "/data/rwchain-all/round2"
BEH_DIR = os.path.join(PROJECT_DIR, "rwchain-beh/data")
EEG_DIR = os.path.join(PROJECT_DIR, "rwchain-eeg")
ALL_BEH_DIR = os.path.join(PROJECT_DIR, 'rwchain-beh', 'combined')


In [ ]:
CODE_DIR = "/home/jenny/evidence-chain/"
FIG_DIR = os.path.join(CODE_DIR, "figs/cnn_kernel_check/")
SAVE_DIR = os.path.join(CODE_DIR, "simple_cnn_models/")

In [ ]:

# Check if the directory exists
if not os.path.exists(FIG_DIR):
    # Create the directory
    os.makedirs(FIG_DIR)
    print(f"Directory '{FIG_DIR}' was created.")
else:
    print(f"Directory '{FIG_DIR}' already exists.")



if not os.path.exists(SAVE_DIR):
    # Create the directory
    os.makedirs(SAVE_DIR)
    print(f"Directory '{SAVE_DIR}' was created.")
else:
    print(f"Directory '{SAVE_DIR}' already exists.")

In [ ]:
stimdur = '250'
pos=9
evaluate_mode = False

In [ ]:

list_of_subj = os.listdir(BEH_DIR)
list_of_subj.sort()
# list_of_subj = [list_of_subj[5]]

df = pd.read_pickle(os.path.join(ALL_BEH_DIR, 'all_df_concat.pkl'))
# organize some columsn
 
df['key'][df['key'] == '[5]'] = 1
df['key'][df['key'] == '[3]'] = 1
df['key'][df['key'] == '[2]'] = 0
df['cumsum'] = df['sequence_clean'].apply(lambda x: [sum(x[:i+1]) for i in range(len(x))])

# get rid of a ;pw acc subject
df = df[df['sid']!='s108']

if stimdur == '100':
    df = df[df['stimDur'] == 0.1]
if stimdur == '250':
    df = df[df['stimDur'] == 0.25]
# df = df[df['stimDur'] != 0.5]
# df = df[df['stimDur'] != 0.05]
# df = df[df['stimDur'] == 0.1]



In [ ]:
# correct the count
df = correct_samples_by_condition(df)

cumsum = np.array(df['cumsum'].to_list())
sequence =  np.array(df['sequence_clean'].to_list())
count = df['count_corrected'].to_numpy()

In [ ]:
df.shape

In [ ]:
df.head()


In [ ]:
count.max()

In [ ]:
# create a matrix of +1 and -1 with NaNs if the chain terminates

chain_matrix = get_chain_matrix(sequence, count)


In [ ]:
# fig, ax = plot_performance_matrix(performance_matrix_test)

In [ ]:
dataset, dataset_resp = make_dataset(pos, count, chain_matrix)
x_train, x_test, y_train, y_test = train_test_split(
    dataset, dataset_resp, test_size=0.3, random_state=2024
)
x_train, x_val, y_train, y_val = train_test_split(
x_train, y_train, test_size=0.25, random_state=1
)

In [ ]:
x_train.shape

In [ ]:
# x_train = get_evidence(x_train)
# x_val = get_evidence(x_val)
# x_test = get_evidence(x_test)

In [ ]:


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert numpy arrays to PyTorch tensors and create datasets
def create_dataset(x, y):
    x_tensor = torch.Tensor(x).unsqueeze(1)  # Add channel dimension
    y_tensor = torch.Tensor(y).unsqueeze(1)
    y_tensor = y_tensor.type(torch.float32)  # Convert boolean labels to float for BCELoss
    return TensorDataset(x_tensor, y_tensor)

# Create datasets
train_dataset = create_dataset(x_train, y_train)
val_dataset = create_dataset(x_val, y_val)
test_dataset = create_dataset(x_test, y_test)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)



x_train.shape

In [ ]:

# Define the model
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 1, kernel_size=4, stride=1, padding=0)  # output is N x 16 x 5
        self.fc1 = nn.Linear(1, 1,bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        # x = torch.relu(x)
        x, max_ind= torch.max(x[:,:,-2:], 2)  # Global average pooling
        # print(max_ind)
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# Training and Evaluation Loops
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        # model.fc1.apply(clipper)
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    all_targets = []
    all_outputs = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            all_outputs.extend(outputs.squeeze().cpu().numpy())
            all_targets.extend(targets.squeeze().cpu().numpy())
    return total_loss / len(dataloader), all_outputs, all_targets


In [ ]:

if evaluate:
    PATH = f'/home/jenny/evidence-chain/simple_cnn_models/best_pos{pos}_running.pkl'
    model =CNN1D()
    model.load_state_dict(torch.load(PATH))
    model.eval()
    model = model.to(device)


    # Loss function and optimizer
    criterion = nn.BCELoss()


    # Train the model
    num_epochs = 20
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        train_loss,_,_ = evaluate(model, train_loader, criterion)
        val_loss, _, _ = evaluate(model, val_loader, criterion)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        # if epoch % 10 ==0:
        #     print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}')

    # Evaluate AUC on the test set
    test_loss, test_outputs, test_targets = evaluate(model, test_loader, criterion)
    train_loss, train_outputs, train_targets = evaluate(model, train_loader, criterion)

    test_auc = roc_auc_score(test_targets, test_outputs)
    train_auc = roc_auc_score(train_targets, train_outputs)


    print(f'Train Loss: {train_loss}')
    print(f'Train AUC: {train_auc}')



    test_auc = roc_auc_score(test_targets, test_outputs)
    print(f'Test Loss: {test_loss}')
    print(f'Test AUC: {test_auc}')


In [20]:
if not evaluate_mode:
    best_test_auc = 0
    best_train_auc = 0
    for i in range(300):
        model = CNN1D().to(device)
        # Loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = Adam(model.parameters(), lr=0.001)



        # Train the model
        num_epochs = 300
        train_losses = []
        val_losses = []

        for epoch in range(num_epochs):
            train_loss = train_epoch(model, train_loader, criterion, optimizer)
            val_loss, _, _ = evaluate(model, val_loader, criterion)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            # if epoch % 10 ==0:
            #     print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}')


        # Evaluate AUC on the test set
        test_loss, test_outputs, test_targets = evaluate(model, test_loader, criterion)
        train_loss, train_outputs, train_targets = evaluate(model, train_loader, criterion)

        test_auc = roc_auc_score(test_targets, test_outputs)
        train_auc = roc_auc_score(train_targets, train_outputs)

        print(f"===================ATTEMP {i}======================")

        print(f'Train Loss: {train_loss}')
        print(f'Train AUC: {train_auc}')



        test_auc = roc_auc_score(test_targets, test_outputs)
        print(f'Test Loss: {test_loss}')
        print(f'Test AUC: {test_auc}')

        if test_auc >best_test_auc and train_auc>best_train_auc:
            best_test_auc = test_auc
            best_train_auc = train_auc
            PATH = f'/home/jenny/evidence-chain/simple_cnn_models/best_pos{pos}_running.pkl'
            torch.save(model.state_dict(), PATH)
            print("saved a better model!\n")

            plt.figure(figsize=(10, 5))
            plt.plot(train_losses, label='Training Loss')
            plt.plot(val_losses, label='Validation Loss')
            plt.title('Training and Validation Loss')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()

            plt.show()
        

In [ ]:
model.fc1.weight.data

In [ ]:
conv_bias = model.conv1.bias.data

In [ ]:
conv_bias

In [ ]:
weights = model.conv1.weight.data

In [ ]:
weights

In [ ]:
fc_weights = model.fc1.weight.data

In [ ]:
fc_weights

In [ ]:
for w in weights:
    w = w.cpu().numpy().squeeze()
    print(w)
    plt.plot(np.arange(0,4),w,'o-')
    plt.show()

In [ ]:
pos

In [ ]:
kernel = weights.cpu().numpy()
position = pos
fc_bias = fc_weights.cpu().numpy()
conv_bias = conv_bias.cpu().numpy()
mydict = {}
mykeys = ["kernel", "position","fc_bias", "conv_bias"]
values = [kernel, position, fc_bias, conv_bias]
for key, value in zip(mykeys, values):
    # Here you can add conditions or transformations
    mydict[key] = value




# Save the dictionary into a pickle file
FILEPATH  = os.path.join(SAVE_DIR, f"pos{pos}_param.pkl")
with open(FILEPATH, 'wb') as pickle_file:
    pickle.dump(mydict, pickle_file)

In [ ]:
mydict

In [ ]:
PATH = f'/home/jenny/evidence-chain/simple_cnn_models/best_pos{pos},pkl'

# torch.save(model.state_dict(), PATH)
# m =CNN1D()
# m.load_state_dict(torch.load(PATH))
# m.eval()

In [ ]:
m = nn.Conv1d(1, 1, 3, stride=2,bias=False)


In [ ]:
input = torch.tensor([1,2,3],dtype=torch.float32).reshape(1,3)

In [ ]:
m.weight.data

In [ ]:
output = m(input)

In [ ]:
output

In [ ]:
m.weight.data @ input.T

In [ ]:
a = np.array((0.4437, -0.3275, -0.3687))
b = np.array((1,2,3))

In [ ]:
pos